In [8]:
url = "https://raw.githubusercontent.com/chiarorosa/ia_aprendizado_maquina_basico/main/ml-dataset/kaggle-basico/attrition_Rate_Company.csv"

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # Avaliação de Acurácia
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [9]:
df = pd.read_csv(url)
df.head()

df.dropna(inplace=True)

labelEnconder = LabelEncoder()
categorizacaoColunas = ['BusinessTravel', 'EducationField', 'Department', 'Gender', 'JobRole', 'MaritalStatus','Attrition','Over18']

for coluna in categorizacaoColunas:
    df[coluna] = labelEnconder.fit_transform(df[coluna])
    
X = df.drop('Attrition', axis=1)
y = df['Attrition']







In [10]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)


# Dividir em dois conjuntos, treinamento e teste, usando pareto 80/20
X_train, X_test, y_train, y_test = train_test_split(
    x_scaled,
    y,
    test_size=0.2, # pareto
    random_state=42 # reproduzivel
)

In [11]:
grade = {
    'loss': ['hinge', 'squared_hinge'],
    'dual': [True, False],
    'tol': np.logspace(-6, -1, 6),  
    'C': np.logspace(-3, 3, 7),  
    'multi_class': ['ovr', 'crammer_singer'],
    'fit_intercept': [True, False],
    'intercept_scaling': np.linspace(1, 10, 10), 
    'class_weight': ['balanced', None],
    'verbose': [0, 1],
}
#Criar verificações sobre penalty e loss

linearSVM = RandomizedSearchCV(
    LinearSVC(),
    grade,
    cv=5,
    scoring='accuracy',
    n_iter=20,  # Número de iterações de busca aleatória
    n_jobs=-1,  # Use todos os núcleos disponíveis
    random_state=42  # Para reprodutibilidade
)

linearSVM.fit(X_train, y_train)

melhor = linearSVM.best_estimator_

y_pred = melhor.predict(X_test)

acuracia = accuracy_score(y_test, y_pred)

print(f'Acurácia: {acuracia * 100:.2f}')
print(f'Melhor modelo: {melhor}')
print(f'Parâmetros do melhor modelo: {linearSVM.best_params_}')

guardaAcc = 0

with open('Melhor Acurracia LinearSVM.txt', 'r+', encoding='utf-8') as f:
        try:
            linha = f.readline().strip()
            if linha.startswith('Acurácia:'):
                guardaAcc = float(linha.split(': ')[1]) / 100
                print(f'Acurácia anterior: {guardaAcc * 100:.2f}')
                if acuracia > guardaAcc:
                    f.seek(0) # vai pro inicio
                    f.write(f'Acurácia: {acuracia * 100:.2f} \n')
                    
                    f.truncate() #  remove residuos antigos
                    print(f'Acurácia atualizada: {acuracia * 100:.2f}')
                else:
                    print('Acurácia não atualizada')
        except Exception as e:
            print(f"Erro ao ler o arquivo: {e}")
       


c:\Users\Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
15 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Pedro\AppDa

[LibLinear]Acurácia: 83.37
Melhor modelo: LinearSVC(C=100.0, dual=True, intercept_scaling=10.0, loss='hinge',
          multi_class='crammer_singer', tol=0.1, verbose=1)
Parâmetros do melhor modelo: {'verbose': 1, 'tol': 0.1, 'multi_class': 'crammer_singer', 'loss': 'hinge', 'intercept_scaling': 10.0, 'fit_intercept': True, 'dual': True, 'class_weight': None, 'C': 100.0}
Acurácia anterior: 83.37
Acurácia atualizada: 83.37


c:\Users\Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
